In [1]:
!pip install pandas networkx scikit-learn torch torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      Traceback (most recent call last):
        File "/Users/firaterman/Documents/fer/research/.venv/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
          main()
        File "/Users/firaterman/Documents/fer/research/.venv/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Users/firaterman/Documents/fer/research

In [2]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.2.0+cpu.html

Looking in links: https://data.pyg.org/whl/torch-2.2.0+cpu.html
  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      Traceback (most recent call last):
        File "/Users/firaterman/Documents/fer/research/.venv/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
          main()
        File "/Users/firaterman/Documents/fer/research/.venv/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Users/firaterman/Documents/fer/research/.venv/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_pro

In [3]:
import torch
print(torch.__version__)

2.2.2


✅ What this script does

✅ Creates a transaction graph from your example.
✅ Assigns example node features.
✅ Builds a PyTorch Geometric Data object.
✅ Defines and trains a small GraphSAGE GNN.
✅ Prints all key tables (features, edge index, final suspicion scores).

In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
import pandas as pd
import networkx as nx
from sklearn.preprocessing import LabelEncoder

# ---------------------------------
# Step 1: Create transactions data
# ---------------------------------
df = pd.DataFrame({
    'from_account': ["A", "A", "B", "C"],
    'to_account':   ["B", "C", "D", "D"],
    'amount': [500, 200, 700, 300]
})

print("🔎 Transactions table:")
print(df, '\n')

# ---------------------------------
# Step 2: Create graph and account list
# ---------------------------------
accounts = pd.unique(df[['from_account', 'to_account']].values.ravel())
account_df = pd.DataFrame({'account': accounts})

# Assign dummy node features
account_df['avg_amount'] = [350, 500, 200, 0]  # Placeholder features
account_df['num_tx'] = [2, 1, 1, 0]
account_df['risk_score'] = [0.2, 0.3, 0.3, 0.9]

print("📄 Account features before encoding:")
print(account_df, '\n')

# Encode account names to indices
encoder = LabelEncoder()
account_df['account_idx'] = encoder.fit_transform(account_df['account'])

# Build edge index
edge_index = torch.tensor(
    df[['from_account', 'to_account']]
    .replace(dict(zip(encoder.classes_, encoder.transform(encoder.classes_))))
    .values.T,
    dtype=torch.long
)

# Create node feature matrix
x = torch.tensor(account_df[['avg_amount', 'num_tx', 'risk_score']].values, dtype=torch.float)

# Labels (1 = suspicious, 0 = normal)
labels = torch.tensor([0, 0, 0, 1], dtype=torch.long)  # D is suspicious

print("🟢 Node features matrix (x):")
print(x, '\n')

print("🔗 Edge index:")
print(edge_index, '\n')

# ---------------------------------
# Step 3: Create PyG data object
# ---------------------------------
data = Data(x=x, edge_index=edge_index, y=labels)

# ---------------------------------
# Step 4: Define a simple GraphSAGE model
# ---------------------------------
class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = SAGEConv(3, 8)  # input features 3 → hidden 8
        self.conv2 = SAGEConv(8, 2)  # hidden 8 → output classes 2

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

model = GNN()

# ---------------------------------
# Step 5: Train the GNN
# ---------------------------------
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

model.train()
for epoch in range(50):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# ---------------------------------
# Step 6: Get predictions and scores
# ---------------------------------
model.eval()
out = model(data.x, data.edge_index)
probs = F.softmax(out, dim=1)[:, 1].detach().numpy()

account_df['suspicion_score'] = probs

print("\n✅ Final account table with suspicion scores:")
print(account_df[['account', 'suspicion_score']])

ModuleNotFoundError: No module named 'torch_geometric'